# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL
from pyspark.sql import types as T

# Crie uma instância da classe AlinharETL

In [2]:
bucket = "bronze"
datamart = "mtecbo"
extrator_bronze = AlinharETL(bucket,datamart)

2024-10-15 10:27:58,461 - INFO - Iniciando Sessão Spark.


In [3]:
schema = T.StructType([
    T.StructField("CODIGO", T.StringType(), True),
    T.StructField("TITULO", T.StringType(), True)
])

# Parâmetros para leitura do arquivo

In [4]:
ano_mes = extrator_bronze.encontrar_ano_mes("mte/cbo")
local = f"s3a://landing/mte/cbo/{ano_mes}/CBO2002 - SubGrupo.csv"

In [5]:
config_adicional = {
    'header': True,
    'sep': ';',
    'encoding': 'iso-8859-1'
}

# Leitura do arquivo

In [6]:
df_compliance_cbo_sub_grupo = extrator_bronze.criar_view_temporaria_arquivo(local, 'compliance_cbo_sub_grupo', schema,**config_adicional)

2024-10-15 10:28:02,650 - INFO - Lista de yaml carregada com sucesso
2024-10-15 10:28:06,016 - INFO - Criando view temporaria.


In [7]:
sql_query = """
    SELECT
        CODIGO AS cd,
        TITULO,
        SUBSTRING(codigo, 1, 2) AS cd_subgrupo_principal
    FROM compliance_cbo_sub_grupo
"""

In [8]:
df_compliance_cbo_sub_grupo = extrator_bronze.carregar_dados_delta(sql_query)

2024-10-15 10:28:06,026 - INFO - Aguarde... Executando Query (Delta)
2024-10-15 10:28:06,027 - INFO - 
    SELECT
        CODIGO AS cd,
        TITULO,
        SUBSTRING(codigo, 1, 2) AS cd_subgrupo_principal
    FROM compliance_cbo_sub_grupo

2024-10-15 10:28:06,239 - INFO - Query (Delta) executada com sucesso


# Gravação no datalake

In [9]:
extrator_bronze.caminho_tabela_delta = 's3a://bronze/mte/cbosubgrupo'

In [10]:
extrator_bronze.salvar_delta(df_compliance_cbo_sub_grupo, 'overwrite')

2024-10-15 10:28:06,250 - INFO - Aguarde... Persistindo dados (overwrite)
2024-10-15 10:28:19,447 - INFO - Dados persistidos com sucesso
2024-10-15 10:28:19,448 - INFO - s3a://bronze/mte/cbosubgrupo
2024-10-15 10:28:19,449 - INFO - Aguarde... Realizando optimize
2024-10-15 10:28:27,478 - INFO - Optimize executado com sucesso.
2024-10-15 10:28:27,479 - INFO - Aguarde... Realizando vacuum
2024-10-15 10:28:51,149 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [11]:
extrator_bronze.parar_sessao()

2024-10-15 10:28:51,536 - INFO - Sessão Spark finalizada.
